In [ ]:
import numpy as np
import random
import networkx as nx
from scipy.spatial import KDTree
import matplotlib.pyplot as plt

In [ ]:
english = np.load('/content/drive/MyDrive/languages/english_newlit_cbow_3_dict.npy', allow_pickle=True)[()]
french = np.load('/content/drive/MyDrive/languages/french_newlit_cbow_3_dict.npy', allow_pickle=True)[()]
german = np.load('/content/drive/MyDrive/languages/german_newlit_cbow_3_dict.npy', allow_pickle=True)[()]
russian = np.load('/content/drive/MyDrive/languages/russian_nofraglit_cbow_3_dict.npy', allow_pickle=True)[()]
vietnamese = np.load('/content/drive/MyDrive/languages/vietnamese_oldlit_SVD_dict.npy', allow_pickle=True)[()]

In [ ]:
print("english: ", len(english))
print("english: ", len(french))
print("english: ", len(german))
print("english: ", len(russian))
print("english: ", len(vietnamese))

english:  153058
english:  510936
english:  1425674
english:  364473
english:  53688


In [ ]:
random.seed(42)


sample_size = int(500)
sampled_items = random.sample(list(english.items()), sample_size)


sampled_arrays = [value for key, value in sampled_items]


english_sample = np.vstack(sampled_arrays)


print(english_sample.shape)

(500, 3)


In [ ]:
def rips_vietoris_graph(points, r):
    tree = KDTree(points)
    edges = list(tree.query_pairs(r))
    G = nx.Graph()
    G.add_nodes_from(range(len(points)))
    G.add_edges_from(edges)
    return G

In [ ]:
G_english = rips_vietoris_graph(english_sample, 0.526)

In [ ]:
random.seed(42)


sample_size = int(500)
sampled_items = random.sample(list(french.items()), sample_size)


sampled_arrays = [value for key, value in sampled_items]


french_sample = np.vstack(sampled_arrays)


print(french_sample.shape)

(500, 3)


In [ ]:
G_french = rips_vietoris_graph(french_sample, 0.526)

In [ ]:
random.seed(42)


sample_size = int(500)
sampled_items = random.sample(list(german.items()), sample_size)


sampled_arrays = [value for key, value in sampled_items]

german_sample = np.vstack(sampled_arrays)


print(german_sample.shape)

(500, 3)


In [ ]:
G_german = rips_vietoris_graph(german_sample, 0.526)

In [ ]:
random.seed(42)


sample_size = int(500)
sampled_items = random.sample(list(russian.items()), sample_size)


sampled_arrays = [value for key, value in sampled_items]


russian_sample = np.vstack(sampled_arrays)


print(russian_sample.shape)

(500, 3)


In [ ]:
G_russian = rips_vietoris_graph(russian_sample, 0.526)

In [ ]:
from collections import defaultdict
import pickle
from BAPG import *
import torch
import time
import os
import argparse

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
def defaul():
    return defaultdict(list)


def BAPG_torch(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        #a = torch.ones([A.shape[0], 1], dtype=A.dtype).cuda()/A.shape[0]
        a = torch.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        #b = torch.ones([B.shape[0], 1], dtype=A.dtype).cuda()/B.shape[0]
        b = torch.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, obj1_list, obj2_list, gap_list = [], [], [], []
    for ii in range(epoch):
        X = X + 1e-10
        X = torch.exp(A@X@B/rho)*X
        pi = X * (a / (X @ torch.ones_like(b)))
        X = torch.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ torch.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            objective = -torch.trace(A @ X @ B @ X.T).item()
            X2 = (X+pi)/2
            gap = (X2.sum(0)-b.squeeze(-1)).norm() + (X2.sum(1)-a.squeeze(-1)).norm()
            gap_list.append(gap.item())
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                obj_list.append(objective)
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list


def BAPG_numpy(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        a = np.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        b = np.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, gap_list = [], []
    for ii in range(epoch):
        X = X+1e-10
        X = np.exp(A@X@B/rho)*X
        pi = X * (a / (X @ np.ones_like(b)))
        X = np.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ np.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            X2 = (X+pi)/2
            gap = np.linalg.norm((X2.sum(0)-b.squeeze(-1))) + np.linalg.norm(X2.sum(1)-a.squeeze(-1))
            gap_list.append(gap.item())
            objective = -np.trace(A @ X @ B @ X.T)
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
database = "synthetic"
rho_list = [0.9, 0.5, 0.2, 0.1, 0.05, 0.01, 0.001]

# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str, default='proteins', help='proteins / reddit / enzymes / synthetic')
# parser.add_argument('--noise_level', type=float, default=0.)
# parser.add_argument('--use_gpu', type=bool, default=False)
# parser.add_argument('--loss_fun', type=str, default='square_loss', help='square_loss/kl_loss')
# parser.add_argument('--rho', type=float, default=[0.5, 0.2, 0.1, 0.05, 0.01], nargs='+')

# args = parser.parse_args()
# rho_list = [0.5, 0.2, 0.1, 0.05, 0.01]
# database = "synthetic"
# noise_level = args.noise_level

# if database == 'proteins':
#     print('------------------Node Matching on PROTIENS---------------')
#     with open('data/PROTEINS/matching.pk', 'rb') as f:
#         graphs, _ = pickle.load(f)

# if database == 'reddit':
#     print('------------------Node Matching on REDDIT---------------')
#     with open('data/REDDIT-BINARY/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)[:500]

# if database == 'enzymes':
#     print('------------------Node Matching on ENZYMES---------------')
#     with open('data/ENZYMES/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)


#ppppppppppppppppp

# if database == 'synthetic':
#     graphs, noise_graphs = [], []
#     print('------------------Node Matching on Synthetic Database---------------')
#     with open('graph1.pk', 'rb') as f:
#         graph_pairs = pickle.load(f)
#         print(graph_pairs)
#         for num_node in [100]:
#             for noise_level in [0.1]:
#                 for G, G_noise in graph_pairs[(num_node, noise_level)]:
#                     graphs.append(G)
#                     noise_graphs.append(G_noise)

#ppppppppppppppppp

# if database != 'synthetic':
#     if noise_level > 0:
#         noise_graphs = []
#         for G_src in graphs:
#             G_dst = add_noisy_edges(G_src, noise_level)
#             G_dst = add_noisy_nodes(G_dst, noise_level)
#             noise_graphs.append(G_dst)
#     else:
#         noise_graphs = graphs

#################


# graphs = [ellipse_graph]
# noise_graphs = [rotated_ellipse_graph]

graphs = [G_english]

noise_graphs = [G_french]

noise_level = 0.
#################


gap_rho = defaultdict(list)
acc_rho = defaultdict(list)
time_rho = defaultdict(list)
gap2_rho = defaultdict(list)
acc2_rho = defaultdict(list)
time2_rho = defaultdict(list)
obj_lists = []
for j in range(len(graphs)):
    print(j)
    G = graphs[j]
    G_noise = noise_graphs[j]
    G_adj = nx.to_numpy_array(G).astype(np.float32)
    G_adj_noise = nx.to_numpy_array(G_noise).astype(np.float32)
    #G_adj_gpu = torch.tensor(G_adj).cuda()
    G_adj_gpu = torch.tensor(G_adj)
    #G_adj_noise_gpu = torch.tensor(G_adj_noise).cuda()
    G_adj_noise_gpu = torch.tensor(G_adj_noise)
    m, n = G_adj.shape[0], G_adj_noise.shape[0]
    for rho in rho_list:  #0.5,0.2,0.1,0.05,
        epoch = 2000 if rho < 0.2 else 4000
        start = time.time()
        # coup_bap, obj_list, gap_list = BAPG_numpy(
        #     A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5, scaling=1.01, max_rho=0.5)
        if True:
            coup_bap, obj_list1, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, epoch=epoch, rho=rho, eps=1e-5)
            end = time.time()
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print('100', gap_list[-1], acc)
        else:
            coup_bap, obj_list1, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5)
            end = time.time()
            print("fffff")
            print(obj_list1)
            print("fffff")
            acc = node_correctness(coup_bap, np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap, np.eye(m))
            print('100', gap_list[-1], acc)

        acc2_rho[rho].append(acc)
        time2_rho[rho].append(end-start)
        gap2_rho[rho].append(gap_list[-1])


for rho in rho_list:
    print('rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}'.format(
        rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
        np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))

    with open('result.txt', 'a+') as f:
        f.write('Data:{},Noise:{},rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}\n'.format(
            database, noise_level, rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
            np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))
print('-'*10)
print(type(G), type(G_noise), sep="    ")
print('-'*3)
print(type(G_adj), type(G_adj_noise), sep="    ")
print('-'*10)

import matplotlib.pyplot as plt

# nx.draw_networkx(G, with_labels=True, node_size=10, pos=nx.spring_layout(G), font_size=8, width=0.1)
# plt.savefig("G.png")
# nx.draw_networkx(G_noise, with_labels=True, node_size=10, font_size=8, width=0.1)
# plt.savefig("G_noise.png")



print('-'*10)
print(obj_lists)

0
iter:1899, smaller than eps
0.9 0.003985538147389889 0.0
100 0.00011775994062190875 0.0
iter:699, smaller than eps
0.5 0.005888083484023809 0.002
100 0.00010958898928947747 0.002
iter:1199, smaller than eps
0.2 0.009756674990057945 0.002
100 0.00011081965931225568 0.0
iter:1099, smaller than eps
0.1 0.013679437339305878 0.004
100 7.909545092843473e-05 0.004
iter:999, smaller than eps
0.05 0.018839208409190178 0.002
100 6.21884100837633e-05 0.002
iter:549, smaller than eps
0.01 0.7001632452011108 0.002
100 5.635307024931535e-05 0.0
0.001 nan 0.002
100 nan 0.002
rho:0.9,gap:3.99e-03,acc:0.00000,time:35.8,gap2:1.18e-04,acc2:0.00000,time2:37.5
rho:0.5,gap:5.89e-03,acc:0.20000,time:13.1,gap2:1.10e-04,acc2:0.20000,time2:14.7
rho:0.2,gap:9.76e-03,acc:0.20000,time:22.8,gap2:1.11e-04,acc2:0.00000,time2:24.5
rho:0.1,gap:1.37e-02,acc:0.40000,time:19.6,gap2:7.91e-05,acc2:0.40000,time2:21.9
rho:0.05,gap:1.88e-02,acc:0.20000,time:18.9,gap2:6.22e-05,acc2:0.20000,time2:20.6
rho:0.01,gap:7.00e-01,acc

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
def defaul():
    return defaultdict(list)


def BAPG_torch(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        #a = torch.ones([A.shape[0], 1], dtype=A.dtype).cuda()/A.shape[0]
        a = torch.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        #b = torch.ones([B.shape[0], 1], dtype=A.dtype).cuda()/B.shape[0]
        b = torch.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, obj1_list, obj2_list, gap_list = [], [], [], []
    for ii in range(epoch):
        X = X + 1e-10
        X = torch.exp(A@X@B/rho)*X
        pi = X * (a / (X @ torch.ones_like(b)))
        X = torch.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ torch.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            objective = -torch.trace(A @ X @ B @ X.T).item()
            X2 = (X+pi)/2
            gap = (X2.sum(0)-b.squeeze(-1)).norm() + (X2.sum(1)-a.squeeze(-1)).norm()
            gap_list.append(gap.item())
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                obj_list.append(objective)
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list


def BAPG_numpy(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        a = np.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        b = np.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, gap_list = [], []
    for ii in range(epoch):
        X = X+1e-10
        X = np.exp(A@X@B/rho)*X
        pi = X * (a / (X @ np.ones_like(b)))
        X = np.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ np.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            X2 = (X+pi)/2
            gap = np.linalg.norm((X2.sum(0)-b.squeeze(-1))) + np.linalg.norm(X2.sum(1)-a.squeeze(-1))
            gap_list.append(gap.item())
            objective = -np.trace(A @ X @ B @ X.T)
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
database = "synthetic"
rho_list = [0.9, 0.5, 0.2, 0.1, 0.05, 0.01, 0.001]

# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str, default='proteins', help='proteins / reddit / enzymes / synthetic')
# parser.add_argument('--noise_level', type=float, default=0.)
# parser.add_argument('--use_gpu', type=bool, default=False)
# parser.add_argument('--loss_fun', type=str, default='square_loss', help='square_loss/kl_loss')
# parser.add_argument('--rho', type=float, default=[0.5, 0.2, 0.1, 0.05, 0.01], nargs='+')

# args = parser.parse_args()
# rho_list = [0.5, 0.2, 0.1, 0.05, 0.01]
# database = "synthetic"
# noise_level = args.noise_level

# if database == 'proteins':
#     print('------------------Node Matching on PROTIENS---------------')
#     with open('data/PROTEINS/matching.pk', 'rb') as f:
#         graphs, _ = pickle.load(f)

# if database == 'reddit':
#     print('------------------Node Matching on REDDIT---------------')
#     with open('data/REDDIT-BINARY/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)[:500]

# if database == 'enzymes':
#     print('------------------Node Matching on ENZYMES---------------')
#     with open('data/ENZYMES/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)


#ppppppppppppppppp

# if database == 'synthetic':
#     graphs, noise_graphs = [], []
#     print('------------------Node Matching on Synthetic Database---------------')
#     with open('graph1.pk', 'rb') as f:
#         graph_pairs = pickle.load(f)
#         print(graph_pairs)
#         for num_node in [100]:
#             for noise_level in [0.1]:
#                 for G, G_noise in graph_pairs[(num_node, noise_level)]:
#                     graphs.append(G)
#                     noise_graphs.append(G_noise)

#ppppppppppppppppp

# if database != 'synthetic':
#     if noise_level > 0:
#         noise_graphs = []
#         for G_src in graphs:
#             G_dst = add_noisy_edges(G_src, noise_level)
#             G_dst = add_noisy_nodes(G_dst, noise_level)
#             noise_graphs.append(G_dst)
#     else:
#         noise_graphs = graphs

#################


# graphs = [ellipse_graph]
# noise_graphs = [rotated_ellipse_graph]

graphs = [G_english]

noise_graphs = [G_german]

noise_level = 0.
#################


gap_rho = defaultdict(list)
acc_rho = defaultdict(list)
time_rho = defaultdict(list)
gap2_rho = defaultdict(list)
acc2_rho = defaultdict(list)
time2_rho = defaultdict(list)
obj_lists = []
for j in range(len(graphs)):
    print(j)
    G = graphs[j]
    G_noise = noise_graphs[j]
    G_adj = nx.to_numpy_array(G).astype(np.float32)
    G_adj_noise = nx.to_numpy_array(G_noise).astype(np.float32)
    #G_adj_gpu = torch.tensor(G_adj).cuda()
    G_adj_gpu = torch.tensor(G_adj)
    #G_adj_noise_gpu = torch.tensor(G_adj_noise).cuda()
    G_adj_noise_gpu = torch.tensor(G_adj_noise)
    m, n = G_adj.shape[0], G_adj_noise.shape[0]
    for rho in rho_list:  #0.5,0.2,0.1,0.05,
        epoch = 2000 if rho < 0.2 else 4000
        start = time.time()
        # coup_bap, obj_list, gap_list = BAPG_numpy(
        #     A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5, scaling=1.01, max_rho=0.5)
        if True:
            coup_bap, obj_list1, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, epoch=epoch, rho=rho, eps=1e-5)
            end = time.time()
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print('100', gap_list[-1], acc)
        else:
            coup_bap, obj_list1, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5)
            end = time.time()
            print("fffff")
            print(obj_list1)
            print("fffff")
            acc = node_correctness(coup_bap, np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap, np.eye(m))
            print('100', gap_list[-1], acc)

        acc2_rho[rho].append(acc)
        time2_rho[rho].append(end-start)
        gap2_rho[rho].append(gap_list[-1])


for rho in rho_list:
    print('rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}'.format(
        rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
        np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))

    with open('result.txt', 'a+') as f:
        f.write('Data:{},Noise:{},rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}\n'.format(
            database, noise_level, rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
            np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))
print('-'*10)
print(type(G), type(G_noise), sep="    ")
print('-'*3)
print(type(G_adj), type(G_adj_noise), sep="    ")
print('-'*10)

import matplotlib.pyplot as plt

# nx.draw_networkx(G, with_labels=True, node_size=10, pos=nx.spring_layout(G), font_size=8, width=0.1)
# plt.savefig("G.png")
# nx.draw_networkx(G_noise, with_labels=True, node_size=10, font_size=8, width=0.1)
# plt.savefig("G_noise.png")



print('-'*10)
print(obj_lists)

0
iter:1199, smaller than eps
0.9 0.005885802675038576 0.004
100 0.00010354116966482252 0.004
iter:649, smaller than eps
0.5 0.00895586609840393 0.002
100 9.847397450357676e-05 0.002
iter:949, smaller than eps
0.2 0.014505339786410332 0.0
100 0.00011557472316781059 0.0
iter:1099, smaller than eps
0.1 0.01860576495528221 0.0
100 0.00010548965656198561 0.0
iter:1349, smaller than eps
0.05 0.023914603516459465 0.004
100 7.57519737817347e-05 0.004
iter:399, smaller than eps
0.01 0.7293136119842529 0.002
100 5.023238554713316e-05 0.004
0.001 nan 0.002
100 nan 0.002
rho:0.9,gap:5.89e-03,acc:0.40000,time:22.0,gap2:1.04e-04,acc2:0.40000,time2:23.7
rho:0.5,gap:8.96e-03,acc:0.20000,time:12.4,gap2:9.85e-05,acc2:0.20000,time2:14.0
rho:0.2,gap:1.45e-02,acc:0.00000,time:18.5,gap2:1.16e-04,acc2:0.00000,time2:20.6
rho:0.1,gap:1.86e-02,acc:0.00000,time:20.1,gap2:1.05e-04,acc2:0.00000,time2:21.8
rho:0.05,gap:2.39e-02,acc:0.40000,time:25.6,gap2:7.58e-05,acc2:0.40000,time2:27.3
rho:0.01,gap:7.29e-01,acc:0

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
def defaul():
    return defaultdict(list)


def BAPG_torch(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        #a = torch.ones([A.shape[0], 1], dtype=A.dtype).cuda()/A.shape[0]
        a = torch.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        #b = torch.ones([B.shape[0], 1], dtype=A.dtype).cuda()/B.shape[0]
        b = torch.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, obj1_list, obj2_list, gap_list = [], [], [], []
    for ii in range(epoch):
        X = X + 1e-10
        X = torch.exp(A@X@B/rho)*X
        pi = X * (a / (X @ torch.ones_like(b)))
        X = torch.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ torch.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            objective = -torch.trace(A @ X @ B @ X.T).item()
            X2 = (X+pi)/2
            gap = (X2.sum(0)-b.squeeze(-1)).norm() + (X2.sum(1)-a.squeeze(-1)).norm()
            gap_list.append(gap.item())
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                obj_list.append(objective)
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list


def BAPG_numpy(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        a = np.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        b = np.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, gap_list = [], []
    for ii in range(epoch):
        X = X+1e-10
        X = np.exp(A@X@B/rho)*X
        pi = X * (a / (X @ np.ones_like(b)))
        X = np.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ np.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            X2 = (X+pi)/2
            gap = np.linalg.norm((X2.sum(0)-b.squeeze(-1))) + np.linalg.norm(X2.sum(1)-a.squeeze(-1))
            gap_list.append(gap.item())
            objective = -np.trace(A @ X @ B @ X.T)
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
database = "synthetic"
rho_list = [0.9, 0.5, 0.2, 0.1, 0.05, 0.01, 0.001]

# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str, default='proteins', help='proteins / reddit / enzymes / synthetic')
# parser.add_argument('--noise_level', type=float, default=0.)
# parser.add_argument('--use_gpu', type=bool, default=False)
# parser.add_argument('--loss_fun', type=str, default='square_loss', help='square_loss/kl_loss')
# parser.add_argument('--rho', type=float, default=[0.5, 0.2, 0.1, 0.05, 0.01], nargs='+')

# args = parser.parse_args()
# rho_list = [0.5, 0.2, 0.1, 0.05, 0.01]
# database = "synthetic"
# noise_level = args.noise_level

# if database == 'proteins':
#     print('------------------Node Matching on PROTIENS---------------')
#     with open('data/PROTEINS/matching.pk', 'rb') as f:
#         graphs, _ = pickle.load(f)

# if database == 'reddit':
#     print('------------------Node Matching on REDDIT---------------')
#     with open('data/REDDIT-BINARY/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)[:500]

# if database == 'enzymes':
#     print('------------------Node Matching on ENZYMES---------------')
#     with open('data/ENZYMES/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)


#ppppppppppppppppp

# if database == 'synthetic':
#     graphs, noise_graphs = [], []
#     print('------------------Node Matching on Synthetic Database---------------')
#     with open('graph1.pk', 'rb') as f:
#         graph_pairs = pickle.load(f)
#         print(graph_pairs)
#         for num_node in [100]:
#             for noise_level in [0.1]:
#                 for G, G_noise in graph_pairs[(num_node, noise_level)]:
#                     graphs.append(G)
#                     noise_graphs.append(G_noise)

#ppppppppppppppppp

# if database != 'synthetic':
#     if noise_level > 0:
#         noise_graphs = []
#         for G_src in graphs:
#             G_dst = add_noisy_edges(G_src, noise_level)
#             G_dst = add_noisy_nodes(G_dst, noise_level)
#             noise_graphs.append(G_dst)
#     else:
#         noise_graphs = graphs

#################


# graphs = [ellipse_graph]
# noise_graphs = [rotated_ellipse_graph]

graphs = [G_english]

noise_graphs = [G_russian]

noise_level = 0.
#################


gap_rho = defaultdict(list)
acc_rho = defaultdict(list)
time_rho = defaultdict(list)
gap2_rho = defaultdict(list)
acc2_rho = defaultdict(list)
time2_rho = defaultdict(list)
obj_lists = []
for j in range(len(graphs)):
    print(j)
    G = graphs[j]
    G_noise = noise_graphs[j]
    G_adj = nx.to_numpy_array(G).astype(np.float32)
    G_adj_noise = nx.to_numpy_array(G_noise).astype(np.float32)
    #G_adj_gpu = torch.tensor(G_adj).cuda()
    G_adj_gpu = torch.tensor(G_adj)
    #G_adj_noise_gpu = torch.tensor(G_adj_noise).cuda()
    G_adj_noise_gpu = torch.tensor(G_adj_noise)
    m, n = G_adj.shape[0], G_adj_noise.shape[0]
    for rho in rho_list:  #0.5,0.2,0.1,0.05,
        epoch = 2000 if rho < 0.2 else 4000
        start = time.time()
        # coup_bap, obj_list, gap_list = BAPG_numpy(
        #     A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5, scaling=1.01, max_rho=0.5)
        if True:
            coup_bap, obj_list1, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, epoch=epoch, rho=rho, eps=1e-5)
            end = time.time()
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print('100', gap_list[-1], acc)
        else:
            coup_bap, obj_list1, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5)
            end = time.time()
            print("fffff")
            print(obj_list1)
            print("fffff")
            acc = node_correctness(coup_bap, np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap, np.eye(m))
            print('100', gap_list[-1], acc)

        acc2_rho[rho].append(acc)
        time2_rho[rho].append(end-start)
        gap2_rho[rho].append(gap_list[-1])


for rho in rho_list:
    print('rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}'.format(
        rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
        np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))

    with open('result.txt', 'a+') as f:
        f.write('Data:{},Noise:{},rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}\n'.format(
            database, noise_level, rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
            np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))
print('-'*10)
print(type(G), type(G_noise), sep="    ")
print('-'*3)
print(type(G_adj), type(G_adj_noise), sep="    ")
print('-'*10)

import matplotlib.pyplot as plt

# nx.draw_networkx(G, with_labels=True, node_size=10, pos=nx.spring_layout(G), font_size=8, width=0.1)
# plt.savefig("G.png")
# nx.draw_networkx(G_noise, with_labels=True, node_size=10, font_size=8, width=0.1)
# plt.savefig("G_noise.png")



print('-'*10)
print(obj_lists)

0
iter:1099, smaller than eps
0.9 0.005083440337330103 0.002
100 9.908372157951817e-05 0.002
iter:1299, smaller than eps
0.5 0.0067520891316235065 0.002
100 0.00011351292778272182 0.002
iter:499, smaller than eps
0.2 0.01048203930258751 0.0
100 0.00010298530105501413 0.0
iter:549, smaller than eps
0.1 0.013329808600246906 0.002
100 9.29866946535185e-05 0.002
iter:499, smaller than eps
0.05 0.017576543614268303 0.0
100 7.5626747275237e-05 0.0
iter:499, smaller than eps
0.01 0.6991822123527527 0.002
100 5.4526251915376633e-05 0.004
0.001 nan 0.002
100 nan 0.002
rho:0.9,gap:5.08e-03,acc:0.20000,time:21.2,gap2:9.91e-05,acc2:0.20000,time2:23.2
rho:0.5,gap:6.75e-03,acc:0.20000,time:23.1,gap2:1.14e-04,acc2:0.20000,time2:25.8
rho:0.2,gap:1.05e-02,acc:0.00000,time:9.0,gap2:1.03e-04,acc2:0.00000,time2:10.8
rho:0.1,gap:1.33e-02,acc:0.20000,time:10.8,gap2:9.30e-05,acc2:0.20000,time2:12.4
rho:0.05,gap:1.76e-02,acc:0.00000,time:9.9,gap2:7.56e-05,acc2:0.00000,time2:11.6
rho:0.01,gap:6.99e-01,acc:0.20

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
def defaul():
    return defaultdict(list)


def BAPG_torch(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        #a = torch.ones([A.shape[0], 1], dtype=A.dtype).cuda()/A.shape[0]
        a = torch.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        #b = torch.ones([B.shape[0], 1], dtype=A.dtype).cuda()/B.shape[0]
        b = torch.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, obj1_list, obj2_list, gap_list = [], [], [], []
    for ii in range(epoch):
        X = X + 1e-10
        X = torch.exp(A@X@B/rho)*X
        pi = X * (a / (X @ torch.ones_like(b)))
        X = torch.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ torch.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            objective = -torch.trace(A @ X @ B @ X.T).item()
            X2 = (X+pi)/2
            gap = (X2.sum(0)-b.squeeze(-1)).norm() + (X2.sum(1)-a.squeeze(-1)).norm()
            gap_list.append(gap.item())
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                obj_list.append(objective)
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list


def BAPG_numpy(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        a = np.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        b = np.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, gap_list = [], []
    for ii in range(epoch):
        X = X+1e-10
        X = np.exp(A@X@B/rho)*X
        pi = X * (a / (X @ np.ones_like(b)))
        X = np.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ np.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            X2 = (X+pi)/2
            gap = np.linalg.norm((X2.sum(0)-b.squeeze(-1))) + np.linalg.norm(X2.sum(1)-a.squeeze(-1))
            gap_list.append(gap.item())
            objective = -np.trace(A @ X @ B @ X.T)
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
database = "synthetic"
rho_list = [0.9, 0.5, 0.2, 0.1, 0.05, 0.01, 0.001]

# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str, default='proteins', help='proteins / reddit / enzymes / synthetic')
# parser.add_argument('--noise_level', type=float, default=0.)
# parser.add_argument('--use_gpu', type=bool, default=False)
# parser.add_argument('--loss_fun', type=str, default='square_loss', help='square_loss/kl_loss')
# parser.add_argument('--rho', type=float, default=[0.5, 0.2, 0.1, 0.05, 0.01], nargs='+')

# args = parser.parse_args()
# rho_list = [0.5, 0.2, 0.1, 0.05, 0.01]
# database = "synthetic"
# noise_level = args.noise_level

# if database == 'proteins':
#     print('------------------Node Matching on PROTIENS---------------')
#     with open('data/PROTEINS/matching.pk', 'rb') as f:
#         graphs, _ = pickle.load(f)

# if database == 'reddit':
#     print('------------------Node Matching on REDDIT---------------')
#     with open('data/REDDIT-BINARY/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)[:500]

# if database == 'enzymes':
#     print('------------------Node Matching on ENZYMES---------------')
#     with open('data/ENZYMES/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)


#ppppppppppppppppp

# if database == 'synthetic':
#     graphs, noise_graphs = [], []
#     print('------------------Node Matching on Synthetic Database---------------')
#     with open('graph1.pk', 'rb') as f:
#         graph_pairs = pickle.load(f)
#         print(graph_pairs)
#         for num_node in [100]:
#             for noise_level in [0.1]:
#                 for G, G_noise in graph_pairs[(num_node, noise_level)]:
#                     graphs.append(G)
#                     noise_graphs.append(G_noise)

#ppppppppppppppppp

# if database != 'synthetic':
#     if noise_level > 0:
#         noise_graphs = []
#         for G_src in graphs:
#             G_dst = add_noisy_edges(G_src, noise_level)
#             G_dst = add_noisy_nodes(G_dst, noise_level)
#             noise_graphs.append(G_dst)
#     else:
#         noise_graphs = graphs

#################


# graphs = [ellipse_graph]
# noise_graphs = [rotated_ellipse_graph]

graphs = [G_french]

noise_graphs = [G_german]

noise_level = 0.
#################


gap_rho = defaultdict(list)
acc_rho = defaultdict(list)
time_rho = defaultdict(list)
gap2_rho = defaultdict(list)
acc2_rho = defaultdict(list)
time2_rho = defaultdict(list)
obj_lists = []
for j in range(len(graphs)):
    print(j)
    G = graphs[j]
    G_noise = noise_graphs[j]
    G_adj = nx.to_numpy_array(G).astype(np.float32)
    G_adj_noise = nx.to_numpy_array(G_noise).astype(np.float32)
    #G_adj_gpu = torch.tensor(G_adj).cuda()
    G_adj_gpu = torch.tensor(G_adj)
    #G_adj_noise_gpu = torch.tensor(G_adj_noise).cuda()
    G_adj_noise_gpu = torch.tensor(G_adj_noise)
    m, n = G_adj.shape[0], G_adj_noise.shape[0]
    for rho in rho_list:  #0.5,0.2,0.1,0.05,
        epoch = 2000 if rho < 0.2 else 4000
        start = time.time()
        # coup_bap, obj_list, gap_list = BAPG_numpy(
        #     A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5, scaling=1.01, max_rho=0.5)
        if True:
            coup_bap, obj_list1, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, epoch=epoch, rho=rho, eps=1e-5)
            end = time.time()
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print('100', gap_list[-1], acc)
        else:
            coup_bap, obj_list1, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5)
            end = time.time()
            print("fffff")
            print(obj_list1)
            print("fffff")
            acc = node_correctness(coup_bap, np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap, np.eye(m))
            print('100', gap_list[-1], acc)

        acc2_rho[rho].append(acc)
        time2_rho[rho].append(end-start)
        gap2_rho[rho].append(gap_list[-1])


for rho in rho_list:
    print('rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}'.format(
        rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
        np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))

    with open('result.txt', 'a+') as f:
        f.write('Data:{},Noise:{},rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}\n'.format(
            database, noise_level, rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
            np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))
print('-'*10)
print(type(G), type(G_noise), sep="    ")
print('-'*3)
print(type(G_adj), type(G_adj_noise), sep="    ")
print('-'*10)

import matplotlib.pyplot as plt

# nx.draw_networkx(G, with_labels=True, node_size=10, pos=nx.spring_layout(G), font_size=8, width=0.1)
# plt.savefig("G.png")
# nx.draw_networkx(G_noise, with_labels=True, node_size=10, font_size=8, width=0.1)
# plt.savefig("G_noise.png")



print('-'*10)
print(obj_lists)

0
iter:1199, smaller than eps
0.9 0.003964466042816639 0.0
100 0.00014171635848470032 0.0
iter:1599, smaller than eps
0.5 0.0056117926724255085 0.0
100 0.00014000512601342052 0.0
iter:1299, smaller than eps
0.2 0.008394692093133926 0.002
100 9.968025551643223e-05 0.002
iter:799, smaller than eps
0.1 0.012185841798782349 0.004
100 8.160190191119909e-05 0.004
iter:1199, smaller than eps
0.05 0.017920464277267456 0.004
100 6.901637243572623e-05 0.004
iter:549, smaller than eps
0.01 0.7641537189483643 0.002
100 5.330513522494584e-05 0.002
0.001 nan 0.002
100 nan 0.002
rho:0.9,gap:3.96e-03,acc:0.00000,time:21.5,gap2:1.42e-04,acc2:0.00000,time2:23.7
rho:0.5,gap:5.61e-03,acc:0.00000,time:30.8,gap2:1.40e-04,acc2:0.00000,time2:32.4
rho:0.2,gap:8.39e-03,acc:0.20000,time:23.8,gap2:9.97e-05,acc2:0.20000,time2:26.0
rho:0.1,gap:1.22e-02,acc:0.40000,time:14.7,gap2:8.16e-05,acc2:0.40000,time2:16.4
rho:0.05,gap:1.79e-02,acc:0.40000,time:21.5,gap2:6.90e-05,acc2:0.40000,time2:24.1
rho:0.01,gap:7.64e-01,a

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
def defaul():
    return defaultdict(list)


def BAPG_torch(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        #a = torch.ones([A.shape[0], 1], dtype=A.dtype).cuda()/A.shape[0]
        a = torch.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        #b = torch.ones([B.shape[0], 1], dtype=A.dtype).cuda()/B.shape[0]
        b = torch.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, obj1_list, obj2_list, gap_list = [], [], [], []
    for ii in range(epoch):
        X = X + 1e-10
        X = torch.exp(A@X@B/rho)*X
        pi = X * (a / (X @ torch.ones_like(b)))
        X = torch.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ torch.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            objective = -torch.trace(A @ X @ B @ X.T).item()
            X2 = (X+pi)/2
            gap = (X2.sum(0)-b.squeeze(-1)).norm() + (X2.sum(1)-a.squeeze(-1)).norm()
            gap_list.append(gap.item())
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                obj_list.append(objective)
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list


def BAPG_numpy(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        a = np.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        b = np.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, gap_list = [], []
    for ii in range(epoch):
        X = X+1e-10
        X = np.exp(A@X@B/rho)*X
        pi = X * (a / (X @ np.ones_like(b)))
        X = np.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ np.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            X2 = (X+pi)/2
            gap = np.linalg.norm((X2.sum(0)-b.squeeze(-1))) + np.linalg.norm(X2.sum(1)-a.squeeze(-1))
            gap_list.append(gap.item())
            objective = -np.trace(A @ X @ B @ X.T)
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
database = "synthetic"
rho_list = [0.9, 0.5, 0.2, 0.1, 0.05, 0.01, 0.001]

# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str, default='proteins', help='proteins / reddit / enzymes / synthetic')
# parser.add_argument('--noise_level', type=float, default=0.)
# parser.add_argument('--use_gpu', type=bool, default=False)
# parser.add_argument('--loss_fun', type=str, default='square_loss', help='square_loss/kl_loss')
# parser.add_argument('--rho', type=float, default=[0.5, 0.2, 0.1, 0.05, 0.01], nargs='+')

# args = parser.parse_args()
# rho_list = [0.5, 0.2, 0.1, 0.05, 0.01]
# database = "synthetic"
# noise_level = args.noise_level

# if database == 'proteins':
#     print('------------------Node Matching on PROTIENS---------------')
#     with open('data/PROTEINS/matching.pk', 'rb') as f:
#         graphs, _ = pickle.load(f)

# if database == 'reddit':
#     print('------------------Node Matching on REDDIT---------------')
#     with open('data/REDDIT-BINARY/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)[:500]

# if database == 'enzymes':
#     print('------------------Node Matching on ENZYMES---------------')
#     with open('data/ENZYMES/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)


#ppppppppppppppppp

# if database == 'synthetic':
#     graphs, noise_graphs = [], []
#     print('------------------Node Matching on Synthetic Database---------------')
#     with open('graph1.pk', 'rb') as f:
#         graph_pairs = pickle.load(f)
#         print(graph_pairs)
#         for num_node in [100]:
#             for noise_level in [0.1]:
#                 for G, G_noise in graph_pairs[(num_node, noise_level)]:
#                     graphs.append(G)
#                     noise_graphs.append(G_noise)

#ppppppppppppppppp

# if database != 'synthetic':
#     if noise_level > 0:
#         noise_graphs = []
#         for G_src in graphs:
#             G_dst = add_noisy_edges(G_src, noise_level)
#             G_dst = add_noisy_nodes(G_dst, noise_level)
#             noise_graphs.append(G_dst)
#     else:
#         noise_graphs = graphs

#################


# graphs = [ellipse_graph]
# noise_graphs = [rotated_ellipse_graph]

graphs = [G_french]

noise_graphs = [G_russian]

noise_level = 0.
#################


gap_rho = defaultdict(list)
acc_rho = defaultdict(list)
time_rho = defaultdict(list)
gap2_rho = defaultdict(list)
acc2_rho = defaultdict(list)
time2_rho = defaultdict(list)
obj_lists = []
for j in range(len(graphs)):
    print(j)
    G = graphs[j]
    G_noise = noise_graphs[j]
    G_adj = nx.to_numpy_array(G).astype(np.float32)
    G_adj_noise = nx.to_numpy_array(G_noise).astype(np.float32)
    #G_adj_gpu = torch.tensor(G_adj).cuda()
    G_adj_gpu = torch.tensor(G_adj)
    #G_adj_noise_gpu = torch.tensor(G_adj_noise).cuda()
    G_adj_noise_gpu = torch.tensor(G_adj_noise)
    m, n = G_adj.shape[0], G_adj_noise.shape[0]
    for rho in rho_list:  #0.5,0.2,0.1,0.05,
        epoch = 2000 if rho < 0.2 else 4000
        start = time.time()
        # coup_bap, obj_list, gap_list = BAPG_numpy(
        #     A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5, scaling=1.01, max_rho=0.5)
        if True:
            coup_bap, obj_list1, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, epoch=epoch, rho=rho, eps=1e-5)
            end = time.time()
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print('100', gap_list[-1], acc)
        else:
            coup_bap, obj_list1, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5)
            end = time.time()
            print("fffff")
            print(obj_list1)
            print("fffff")
            acc = node_correctness(coup_bap, np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap, np.eye(m))
            print('100', gap_list[-1], acc)

        acc2_rho[rho].append(acc)
        time2_rho[rho].append(end-start)
        gap2_rho[rho].append(gap_list[-1])


for rho in rho_list:
    print('rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}'.format(
        rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
        np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))

    with open('result.txt', 'a+') as f:
        f.write('Data:{},Noise:{},rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}\n'.format(
            database, noise_level, rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
            np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))
print('-'*10)
print(type(G), type(G_noise), sep="    ")
print('-'*3)
print(type(G_adj), type(G_adj_noise), sep="    ")
print('-'*10)

import matplotlib.pyplot as plt

# nx.draw_networkx(G, with_labels=True, node_size=10, pos=nx.spring_layout(G), font_size=8, width=0.1)
# plt.savefig("G.png")
# nx.draw_networkx(G_noise, with_labels=True, node_size=10, font_size=8, width=0.1)
# plt.savefig("G_noise.png")



print('-'*10)
print(obj_lists)

0
iter:1999, smaller than eps
0.9 0.0015654730377718806 0.004
100 0.00036059663398191333 0.004
iter:699, smaller than eps
0.5 0.0023868137504905462 0.004
100 0.0002942407736554742 0.002
iter:849, smaller than eps
0.2 0.005200449842959642 0.004
100 0.00018080903100781143 0.004
iter:1249, smaller than eps
0.1 0.009510001167654991 0.008
100 0.00013590148591902107 0.008
iter:1299, smaller than eps
0.05 0.015729378908872604 0.004
100 0.00012627022806555033 0.004
iter:349, smaller than eps
0.01 0.7380968332290649 0.002
100 5.6780198065098375e-05 0.004
0.001 nan 0.002
100 nan 0.002
rho:0.9,gap:1.57e-03,acc:0.40000,time:36.9,gap2:3.61e-04,acc2:0.40000,time2:38.5
rho:0.5,gap:2.39e-03,acc:0.40000,time:13.0,gap2:2.94e-04,acc2:0.20000,time2:14.7
rho:0.2,gap:5.20e-03,acc:0.40000,time:15.5,gap2:1.81e-04,acc2:0.40000,time2:17.2
rho:0.1,gap:9.51e-03,acc:0.80000,time:23.3,gap2:1.36e-04,acc2:0.80000,time2:25.0
rho:0.05,gap:1.57e-02,acc:0.40000,time:24.2,gap2:1.26e-04,acc2:0.40000,time2:25.9
rho:0.01,gap

In [57]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
def defaul():
    return defaultdict(list)


def BAPG_torch(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        #a = torch.ones([A.shape[0], 1], dtype=A.dtype).cuda()/A.shape[0]
        a = torch.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        #b = torch.ones([B.shape[0], 1], dtype=A.dtype).cuda()/B.shape[0]
        b = torch.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, obj1_list, obj2_list, gap_list = [], [], [], []
    for ii in range(epoch):
        X = X + 1e-10
        X = torch.exp(A@X@B/rho)*X
        pi = X * (a / (X @ torch.ones_like(b)))
        X = torch.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ torch.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            objective = -torch.trace(A @ X @ B @ X.T).item()
            X2 = (X+pi)/2
            gap = (X2.sum(0)-b.squeeze(-1)).norm() + (X2.sum(1)-a.squeeze(-1)).norm()
            gap_list.append(gap.item())
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                obj_list.append(objective)
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list


def BAPG_numpy(A, B, a=None, b=None, X=None, epoch=200, eps=1e-5, rho=1e-1):
    if a is None:
        a = np.ones([A.shape[0], 1], dtype=A.dtype)/A.shape[0]
    if b is None:
        b = np.ones([B.shape[0], 1], dtype=A.dtype)/B.shape[0]
    if X is None:
        X = a@b.T
    obj_list, gap_list = [], []
    for ii in range(epoch):
        X = X+1e-10
        X = np.exp(A@X@B/rho)*X
        pi = X * (a / (X @ np.ones_like(b)))
        X = np.exp(A@pi@B/rho)*pi
        X = X * (b.T / (X.T @ np.ones_like(a)).T)
        if (ii+1) % 50 == 0:
            X2 = (X+pi)/2
            gap = np.linalg.norm((X2.sum(0)-b.squeeze(-1))) + np.linalg.norm(X2.sum(1)-a.squeeze(-1))
            gap_list.append(gap.item())
            objective = -np.trace(A @ X @ B @ X.T)
            if len(obj_list) > 0 and np.abs((objective-obj_list[-1])/obj_list[-1]) < eps:
                print('iter:{}, smaller than eps'.format(ii))
                break
            obj_list.append(objective)
    return X2, obj_list, gap_list
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
database = "synthetic"
rho_list = [0.9, 0.5, 0.2, 0.1, 0.05, 0.01, 0.001]

# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str, default='proteins', help='proteins / reddit / enzymes / synthetic')
# parser.add_argument('--noise_level', type=float, default=0.)
# parser.add_argument('--use_gpu', type=bool, default=False)
# parser.add_argument('--loss_fun', type=str, default='square_loss', help='square_loss/kl_loss')
# parser.add_argument('--rho', type=float, default=[0.5, 0.2, 0.1, 0.05, 0.01], nargs='+')

# args = parser.parse_args()
# rho_list = [0.5, 0.2, 0.1, 0.05, 0.01]
# database = "synthetic"
# noise_level = args.noise_level

# if database == 'proteins':
#     print('------------------Node Matching on PROTIENS---------------')
#     with open('data/PROTEINS/matching.pk', 'rb') as f:
#         graphs, _ = pickle.load(f)

# if database == 'reddit':
#     print('------------------Node Matching on REDDIT---------------')
#     with open('data/REDDIT-BINARY/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)[:500]

# if database == 'enzymes':
#     print('------------------Node Matching on ENZYMES---------------')
#     with open('data/ENZYMES/matching.pk', 'rb') as f:
#         graphs = pickle.load(f)


#ppppppppppppppppp

# if database == 'synthetic':
#     graphs, noise_graphs = [], []
#     print('------------------Node Matching on Synthetic Database---------------')
#     with open('graph1.pk', 'rb') as f:
#         graph_pairs = pickle.load(f)
#         print(graph_pairs)
#         for num_node in [100]:
#             for noise_level in [0.1]:
#                 for G, G_noise in graph_pairs[(num_node, noise_level)]:
#                     graphs.append(G)
#                     noise_graphs.append(G_noise)

#ppppppppppppppppp

# if database != 'synthetic':
#     if noise_level > 0:
#         noise_graphs = []
#         for G_src in graphs:
#             G_dst = add_noisy_edges(G_src, noise_level)
#             G_dst = add_noisy_nodes(G_dst, noise_level)
#             noise_graphs.append(G_dst)
#     else:
#         noise_graphs = graphs

#################


# graphs = [ellipse_graph]
# noise_graphs = [rotated_ellipse_graph]

graphs = [G_german]

noise_graphs = [G_russian]

noise_level = 0.
#################


gap_rho = defaultdict(list)
acc_rho = defaultdict(list)
time_rho = defaultdict(list)
gap2_rho = defaultdict(list)
acc2_rho = defaultdict(list)
time2_rho = defaultdict(list)
obj_lists = []
for j in range(len(graphs)):
    print(j)
    G = graphs[j]
    G_noise = noise_graphs[j]
    G_adj = nx.to_numpy_array(G).astype(np.float32)
    G_adj_noise = nx.to_numpy_array(G_noise).astype(np.float32)
    #G_adj_gpu = torch.tensor(G_adj).cuda()
    G_adj_gpu = torch.tensor(G_adj)
    #G_adj_noise_gpu = torch.tensor(G_adj_noise).cuda()
    G_adj_noise_gpu = torch.tensor(G_adj_noise)
    m, n = G_adj.shape[0], G_adj_noise.shape[0]
    for rho in rho_list:  #0.5,0.2,0.1,0.05,
        epoch = 2000 if rho < 0.2 else 4000
        start = time.time()
        # coup_bap, obj_list, gap_list = BAPG_numpy(
        #     A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5, scaling=1.01, max_rho=0.5)
        if True:
            coup_bap, obj_list1, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, epoch=epoch, rho=rho, eps=1e-5)
            end = time.time()
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_torch(
                A=G_adj_gpu, B=G_adj_noise_gpu, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap.cpu().numpy(), np.eye(m))
            print('100', gap_list[-1], acc)
        else:
            coup_bap, obj_list1, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, epoch=2000, rho=rho, eps=1e-5)
            end = time.time()
            print("fffff")
            print(obj_list1)
            print("fffff")
            acc = node_correctness(coup_bap, np.eye(m))
            print(rho, gap_list[-1], acc)
            acc_rho[rho].append(acc)
            time_rho[rho].append(end-start)
            gap_rho[rho].append(gap_list[-1])
            coup_bap, obj_list2, gap_list = BAPG_numpy(
                A=G_adj, B=G_adj_noise, X=coup_bap, epoch=100, rho=100, eps=1e-5)
            end = time.time()
            obj_lists.append((obj_list1, obj_list2))
            acc = node_correctness(coup_bap, np.eye(m))
            print('100', gap_list[-1], acc)

        acc2_rho[rho].append(acc)
        time2_rho[rho].append(end-start)
        gap2_rho[rho].append(gap_list[-1])


for rho in rho_list:
    print('rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}'.format(
        rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
        np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))

    with open('result.txt', 'a+') as f:
        f.write('Data:{},Noise:{},rho:{},gap:{:.2e},acc:{:.5f},time:{:.1f},gap2:{:.2e},acc2:{:.5f},time2:{:.1f}\n'.format(
            database, noise_level, rho, np.mean(gap_rho[rho]), np.mean(acc_rho[rho])*100, np.sum(time_rho[rho]),
            np.mean(gap2_rho[rho]), np.mean(acc2_rho[rho])*100, np.sum(time2_rho[rho])))
print('-'*10)
print(type(G), type(G_noise), sep="    ")
print('-'*3)
print(type(G_adj), type(G_adj_noise), sep="    ")
print('-'*10)

import matplotlib.pyplot as plt

# nx.draw_networkx(G, with_labels=True, node_size=10, pos=nx.spring_layout(G), font_size=8, width=0.1)
# plt.savefig("G.png")
# nx.draw_networkx(G_noise, with_labels=True, node_size=10, font_size=8, width=0.1)
# plt.savefig("G_noise.png")



print('-'*10)
print(obj_lists)

0
iter:2099, smaller than eps
0.9 0.0031573474407196045 0.002
100 0.00016901249182410538 0.002
iter:699, smaller than eps
0.5 0.004462127108126879 0.0
100 0.00015142644406296313 0.0
iter:449, smaller than eps
0.2 0.007133902050554752 0.0
100 0.00011220475425943732 0.0
iter:1099, smaller than eps
0.1 0.010953596793115139 0.0
100 7.700316200498492e-05 0.0
iter:549, smaller than eps
0.05 0.016464710235595703 0.002
100 7.035435555735603e-05 0.002
iter:349, smaller than eps
0.01 0.7681795358657837 0.002
100 5.064034485258162e-05 0.002
0.001 nan 0.002
100 nan 0.002
rho:0.9,gap:3.16e-03,acc:0.20000,time:40.4,gap2:1.69e-04,acc2:0.20000,time2:42.1
rho:0.5,gap:4.46e-03,acc:0.00000,time:13.1,gap2:1.51e-04,acc2:0.00000,time2:14.8
rho:0.2,gap:7.13e-03,acc:0.00000,time:9.3,gap2:1.12e-04,acc2:0.00000,time2:11.0
rho:0.1,gap:1.10e-02,acc:0.00000,time:19.8,gap2:7.70e-05,acc2:0.00000,time2:22.6
rho:0.05,gap:1.65e-02,acc:0.20000,time:9.4,gap2:7.04e-05,acc2:0.20000,time2:11.0
rho:0.01,gap:7.68e-01,acc:0.20